In [1]:
## Used for vectorizing the raw data (run it once on train and once on test) :
## Pivoting it from the initial feature_name:feature_value form to a vector
## Translating categoric variables into N-1 dummy variables
## For time series variables (have multiple measurements in different times):
### Calculating slope - the diffs between each measurement and the first measurement (0 day) 
### Reducing the slopes to a single variable - the mean of all slopes 
## Filling empty values with means - NOTE that these have to be the train data means

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../train_data.csv', sep = '|', error_bad_lines=False, index_col=False, dtype='unicode')
df.describe()

,SubjectID,form_name,feature_name,feature_value,feature_unit,feature_delta
count,1138647,1138647,1138647,1138635,727028,1136734
unique,1777,12,1337,36409,48,12952
top,25028,Lab Test,Q2_Salivation,4.0,mmol/L,0.0
freq,1069,615019,18879,83967,169840,74413


In [3]:
df.head()

,SubjectID,form_name,feature_name,feature_value,feature_unit,feature_delta
0,533,Demographic,Gender,F,NaN,0.0
1,533,Demographic,Age,65,NaN,0.0
2,533,Demographic,Race,White,NaN,0.0
3,533,ALSHX,onset_delta,-1023,NaN,0.0
4,533,ALSHX,diag_delta,-44,NaN,0.0


In [4]:
def get_demo(df):
    demographics = df[df.form_name == 'Demographic']
    demo_pivot = pd.pivot_table(demographics, values = ['feature_value'], index = ['SubjectID'], 
                                columns = ['feature_name'], aggfunc = lambda x:x)
    gender = pd.get_dummies(demo_pivot['feature_value']['Gender'])
    gender = gender.drop('M', axis=1)
    race = pd.get_dummies(demo_pivot['feature_value']['Race'])
    race = race.drop('Unknown', axis=1)
    return pd.merge(gender, race, right_index=True, left_index=True)

demo_final = get_demo(df)    
demo_final.head()


,F,Asian,Black,Hispanic,Other,White
SubjectID,,,,,,
100256,0,0,0,0,0,1
100626,0,0,0,0,0,1
100806,0,0,0,0,0,1
100981,0,0,0,0,0,1
101154,0,0,0,0,0,1


In [5]:
### Calculating slope - the diffs between each measurement and the first measurement (0 day) 
def calc_slope(row) :
    time_delta =  (float(row['feature_delta_int_y']) - float(row['feature_delta_int_x']))
    return (row['feature_value_float_y'] - row['feature_value_float_x'])/time_delta

def get_vitals_slope(df):
    vitals = df[df.form_name == 'Vitals']
    # There were duplicate measurements of vitals with the same feature_delta :(
    vitals = vitals.drop_duplicates(subset = ['SubjectID', 'feature_delta', 'feature_name'], take_last=True)
    vitals.loc[:, 'feature_value_float'] = vitals['feature_value'].astype(float)
    vitals.loc[:, 'feature_delta_int'] = vitals['feature_delta'].astype(int)
    vitals = vitals[vitals.feature_delta_int < 92]
    vitals_noinit = vitals[vitals.feature_delta_int != 0]
    vitals_init = vitals[vitals.feature_delta_int == 0]
    vitals_with_init = pd.merge(vitals_noinit, vitals_init, on=['SubjectID','feature_name']) 
    vitals_with_init.loc[:, 'feature_value_slope'] = vitals_with_init.apply(calc_slope, axis=1)
    return vitals_with_init

vitals_slope = get_vitals_slope(df)
vitals_slope.head()

,SubjectID,form_name_x,feature_name,feature_value_x,feature_unit_x,feature_delta_x,feature_value_float_x,feature_delta_int_x,form_name_y,feature_value_y,feature_unit_y,feature_delta_y,feature_value_float_y,feature_delta_int_y,feature_value_slope
0,533,Vitals,bp_diastolic,83.0,mmHg,12,83,12,Vitals,83.0,mmHg,0,83,0,-0.000000
1,533,Vitals,bp_diastolic,69.0,mmHg,25,69,25,Vitals,83.0,mmHg,0,83,0,-0.560000
2,533,Vitals,bp_diastolic,88.0,mmHg,39,88,39,Vitals,83.0,mmHg,0,83,0,0.128205
3,533,Vitals,bp_diastolic,90.0,mmHg,53,90,53,Vitals,83.0,mmHg,0,83,0,0.132075
4,533,Vitals,bp_diastolic,85.0,mmHg,91,85,91,Vitals,83.0,mmHg,0,83,0,0.021978


In [6]:
### Reducing the slopes to a single variable - the mean of all slopes 
def get_vitals_slope_mean(vitals_slope):
    vitals_pivot = pd.pivot_table(vitals_slope, values = ['feature_value_slope'], index = ['SubjectID'], columns = ['feature_name'], aggfunc = 'mean')
    vitals_pivot = vitals_pivot['feature_value_slope']
    # BMI was measured only once, on the 0 day... 
    vitals_pivot = vitals_pivot.drop('BMI', axis=1)
    return vitals_pivot

vitals_slope_mean = get_vitals_slope_mean(vitals_slope)
final_data = pd.merge(vitals_slope_mean, demo_final, right_index=True, left_index=True)
final_data.head()

feature_name,bp_diastolic,bp_systolic,height,pulse,respiratory_rate,temperature,weight,F,Asian,Black,Hispanic,Other,White
SubjectID,,,,,,,,,,,,,
100256,-0.516411,-0.340148,NaN,0.440030,0.009804,0.037261,-0.141928,0,0,0,0,0,1
100626,-0.136655,-0.347385,NaN,0.007043,0.064004,NaN,0.011066,0,0,0,0,0,1
100806,0.436971,0.236458,NaN,-0.292272,0.005522,NaN,0.043708,0,0,0,0,0,1
100981,-0.368421,-0.316349,NaN,0.732923,NaN,NaN,-0.026316,0,0,0,0,0,1
101154,-1.761806,1.258433,NaN,-0.001538,0.266419,NaN,-0.090551,0,0,0,0,0,1


In [7]:
## Filling empty values with means - NOTE that these have to be the train data means
train_data_means = final_data.mean()
final_data = final_data.fillna(train_data_means)
final_data.head()

feature_name,bp_diastolic,bp_systolic,height,pulse,respiratory_rate,temperature,weight,F,Asian,Black,Hispanic,Other,White
SubjectID,,,,,,,,,,,,,
100256,-0.516411,-0.340148,0.00025,0.440030,0.009804,0.037261,-0.141928,0,0,0,0,0,1
100626,-0.136655,-0.347385,0.00025,0.007043,0.064004,-0.000389,0.011066,0,0,0,0,0,1
100806,0.436971,0.236458,0.00025,-0.292272,0.005522,-0.000389,0.043708,0,0,0,0,0,1
100981,-0.368421,-0.316349,0.00025,0.732923,0.005826,-0.000389,-0.026316,0,0,0,0,0,1
101154,-1.761806,1.258433,0.00025,-0.001538,0.266419,-0.000389,-0.090551,0,0,0,0,0,1


In [10]:
for t in ["train", "test"]:
    df = pd.read_csv('../' + t + '_data.csv', sep = '|', error_bad_lines=False, index_col=False, dtype='unicode')
    demo_final = get_demo(df)    
    vitals_slope = get_vitals_slope(df)
    vitals_slope_mean = get_vitals_slope_mean(vitals_slope)
    final_data = pd.merge(vitals_slope_mean, demo_final, right_index=True, left_index=True)
    final_data = final_data.fillna(train_data_means)
    final_data.to_csv('../' + t + '_data_vectorized.csv' ,sep='|')